# Examine overlaps with published data

In [ ]:
import pandas as pd
import numpy as np

## Public si-eQTL analysis

In [ ]:
shen = ["GDAP2", "AIM2", "SLAMF6", "RLF", "ATG4C", "FUT7",
        "TMEM218", "C11orf74", "RAB35", "TMEM5", "HNRNPK",
        "CDCA3", "ERCC5", "GJB6", "SNTB2", "SPNS3", 
        "XAF1", "RBBP8", "RUFY4", "CA2", "RAPGEF1"]
print("Shen et al.:")
print(len(shen))

kukurba = ["NOD2", "WDR36", "BSCL2", "MAP7D3", "RHOXF1", "DNAH1"]
print("Kukurba et al.:")
print(len(kukurba))

yao = ["NOD2", "HLA-DRB5", "HLA-DRB5", "KIAA0586", "PPP2R5A", 
       "TSNAXIP1", "MUT", "GRIK2", "C15orf37", "LIMA1", "IL6ST", 
       "HCG8", "BLOC1S3", "NKX3-1", "CXorf23"]
print("Yao et al.:")
print(len(np.unique(yao)))
print("Total of Yao + Kukurba:")
len(set(yao) | set(kukurba))

## Load BrainSeq si-eQTL results

### Interacting variant-gene pairs

In [ ]:
bs0 = pd.read_csv("../../summary_table/_m/BrainSeq_sexGenotypes_4features_3regions.txt.gz", sep='\t')
bs0["ensembl_gene_id"] = bs0.gene_id.str.replace("\\..*", "", regex=True)
biomart = pd.read_csv("../../../residualized_expression/final_plot_eqtls/_h/biomart.csv", index_col=0)
bs = bs0.merge(biomart, on="ensembl_gene_id").drop_duplicates(subset="gene_id")
print(bs.shape)
bs.tail(2)

In [ ]:
bs[(bs['external_gene_name'].isin(shen))].to_csv("siEQTL_Shen_comparison.csv", index=False)

In [ ]:
bs[(bs['external_gene_name'].isin(kukurba))]

In [ ]:
bs[(bs['external_gene_name'].isin(yao))]

In [ ]:
bs[(bs['external_gene_name'].isin(shen+kukurba+yao))]

### eigenMT corrected p-values

In [ ]:
df0 = pd.read_csv("../../../../summary_table/_m/Brainseq_sex_interacting_4features_3regions.eFeatures.txt.gz", sep='\t')
df = df0[(df0["Type"] == "Gene")].drop_duplicates(subset="gene_id").drop("Tissue", axis=1)
df["ensembl_gene_id"] = df.gene_id.str.replace("\\..*", "", regex=True)
dft = df.merge(biomart, on="ensembl_gene_id")
print(dft.shape)
dft.head(2)

In [ ]:
## Not all genes significant via mashr are also significant with eigenMT
len(set(bs.gene_id) & set(dft.gene_id))

In [ ]:
dft[(dft['external_gene_name'].isin(shen))]

In [ ]:
dft[(dft['external_gene_name'].isin(shen)) & (dft["eigenMT_BH"] < 0.25)]

In [ ]:
dft[(dft['external_gene_name'].isin(kukurba)) & (dft["eigenMT_BH"] < 0.25)]

In [ ]:
dft[(dft['external_gene_name'].isin(yao)) & (dft["eigenMT_BH"] < 0.25)]

## GTEx comparison

In [ ]:
gtex = pd.read_csv("/ceph/users/jbenja13/projects/sex_sz_ria/input/gtex_v8/"+\
                   "_m/GTEx_Analysis_v8_sbeQTLs/GTEx_Analysis_v8_sbeQTLs.txt", 
                   sep='\t')
gtex.iloc[0:2, 0:10]

In [ ]:
gtex.iloc[0:2, 10:14]

In [ ]:
## qval threshold equal to number of published sb-eQTL
gtex[(gtex['qval'] < 0.25) & (gtex["Tissue"].str.contains("Brain"))]\
    .loc[:, ["ensembl_gene_id", "hugo_gene_id", "Tissue", "pvals.corrected", 'qval']].head(10)

In [ ]:
## qval threshold equal to number of published sb-eQTL
gtex[(gtex['qval'] < 0.25) & (gtex["Tissue"].str.contains("Whole"))]\
    .loc[:, ["ensembl_gene_id", "hugo_gene_id", "Tissue", "pvals.corrected", 'qval']].head(10)

In [ ]:
gtex_sig = gtex[(gtex['qval'] < 0.25)]
gtex_sig.shape

In [ ]:
gtex_sig.head(10)

### mashr

In [ ]:
gtex_overlap = bs[(bs['gene_id'].isin(gtex_sig.ensembl_gene_id))].drop_duplicates()
print(gtex_overlap.shape)
gtex_overlap

In [ ]:
gtex_overlap.shape[0]/bs.shape[0] * 100

In [ ]:
gtex_sig[(gtex_sig['ensembl_gene_id'].isin(bs.gene_id))]

In [ ]:
gtex_sig[(gtex_sig['ensembl_gene_id'].isin(bs.gene_id))]\
    .to_csv("siEQTL_gtex_comparison.csv", index=False)

### eigenMT corrected

In [ ]:
gtex_overlap = dft[(dft['gene_id'].isin(gtex_sig.ensembl_gene_id)) & (dft["BF"] < 0.05)].drop_duplicates()
print(gtex_overlap.shape)
gtex_overlap.sort_values("BF").head()

In [ ]:
gtex_overlap = dft[(dft['gene_id'].isin(gtex_sig.ensembl_gene_id))].drop_duplicates()
print(gtex_overlap.shape)
gtex_overlap.sort_values("BF").head()

In [ ]:
gtex_overlap.shape[0]/dft.shape[0] * 100